In [1]:
'''
Author: Mikhail Milchenko, mmilchenko@wustl.edu
Copyright (c) 2021, Computational Imaging Lab, Washington University School of Medicine

Redistribution and use in source and binary forms, for any purpose, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
'''
import argparse, numpy as np
from scipy.spatial.distance import cdist


In [2]:
def expand_3d_contour(contours,dist,imwid, imht):
    '''
    Expand by the given distance a 3D object represented by
    a list of closed 2D contours.
    contours is a list of lists of array-like 3d points
    dim1 <-- x , dim2 <-- y, dim3 <-- z (constant), dim4 <-- point index.
    '''
    c,c_out=contours,[]
    imdepth=len(c)
    
    for z in range (imdepth):    
        c0=np.array(c[z])
        if z>0:
            c_1=np.array(c[z-1])
            d_1=np.argmin(cdist(c0,c_1),1)
        else: 
            d_1=None

        if z<imdepth-1:
            c1=np.array(c[z+1])
            d1=np.argmin(cdist(c0,c1),1)
        else: 
            d1=None

        co=[]
        l=len(c0)
        z0=c0[0][2]

        for ind in range(len(c0)):
            pt_1,pt0,pt1=c0[ ind-1 ],c0[ind],c0[ (ind+1) % l ]
            vl,vr=pt0-pt_1,pt1-pt0
            normals=[]
            if d_1 is not None:
                vd=c_1[d_1[ind]]-pt0
                nl,nr=np.cross(vd,vl),np.cross(vr,vd)
                normals.append(nl/np.linalg.norm(nl))
                normals.append(nr/np.linalg.norm(nr))
            if d1 is not None:
                vu=c1[d1[ind]]-pt0
                nl,nr=np.cross(vl,vu),np.cross(vu,vr)
                normals.append(nl/np.linalg.norm(nl))
                normals.append(nr/np.linalg.norm(nr))

            normal=np.array(normals).sum(0)
            normal/=np.linalg.norm(normal)
            pt_new=pt0+normal*dist
            pt_new[2]=z0 #project to the plane of the old contour
            co.append(pt_new)

        c_out.append(co)        
        
    return c_out    


In [15]:
c=[]
c.append([[2,2,1],[4,2,1],[2,4,1]])
c.append([[2,2,2],[2,4,2],[4,4,2],[4,2,2]])
c.append([[2.1,1.8,3],[2.3,3.8,3],[4.1,3.8,3],[3.5,2.1,3],[3,2,3]])
dist=1.0
imwid,imht=10,10
print ("Input list of contours")
c


Input list of contours


[[[2, 2, 1], [4, 2, 1], [2, 4, 1]],
 [[2, 2, 2], [2, 4, 2], [4, 4, 2], [4, 2, 2]],
 [[2.1, 1.8, 3], [2.3, 3.8, 3], [4.1, 3.8, 3], [3.5, 2.1, 3], [3, 2, 3]]]

In [16]:
c_out=expand_3d_contour(c,dist,imwid, imht)
print("List of output normals")
c_out


List of output normals


[[array([1.29289322, 2.70710678, 1.        ]),
  array([3.61731657, 1.07612047, 1.        ]),
  array([2.92387953, 4.38268343, 1.        ])],
 [array([1.29421825, 2.70066749, 2.        ]),
  array([2.70263278, 4.71079897, 2.        ]),
  array([4.79000749, 3.43461296, 2.        ]),
  array([3.31684897, 1.28056791, 2.        ])],
 [array([1.36390363, 2.44579197, 3.        ]),
  array([3.02824595, 4.48034347, 3.        ]),
  array([4.66589729, 3.00425086, 3.        ]),
  array([2.82690215, 1.41056047, 3.        ]),
  array([3.67756668, 2.28601887, 3.        ])]]